# GameNet (Hartford et al 2016)

## Session

In [31]:
import tensorflow as tf
import numpy as np
import pandas as pd
from __future__ import division
sess = tf.InteractiveSession()

## Data

In [32]:
import_csv = pd.read_csv('gamesmxn.csv')
tmp = np.empty((import_csv.shape[0]))
tmp[:] = np.NAN
for i in range(import_csv.shape[0]):
    if import_csv['shape'][i] == '3 3' and import_csv['symmetric'][i] == 1:
        tmp[i] = i
index = tmp[~np.isnan(tmp)]
inputs_row = np.zeros((index.shape[0], 3, 3, 2))
inputs_col = np.zeros((index.shape[0], 3, 3, 2))
target_row = np.zeros((index.shape[0], 3, 1))
for j in range(index.shape[0]):
    Ur = np.matrix(import_csv['matrixrow'][int(index[j])])
    Ur = (Ur-np.mean(Ur))/np.std(Ur)
    Uc = np.transpose(Ur)
    ar = np.matrix(import_csv['choicerow'][int(index[j])])
    if ar.shape[1] == 2:
        ar = ar[:,0]/ar[:,1]
    inputs_row[j, :, :, 0] = Ur
    inputs_row[j, :, :, 1] = Uc
    inputs_col[j, :, :, 0] = Uc
    inputs_col[j, :, :, 1] = Ur
    target_row[j] = ar

### Training and test sets 

In [33]:
shuffle = np.random.permutation(range(inputs_row.shape[0]))
index_train = shuffle[0:inputs_row.shape[0]//5*4] 
index_tests = shuffle[inputs_row.shape[0]//5*4:inputs_row.shape[0]]
inputs_row_train = inputs_row[index_train]
inputs_col_train = inputs_col[index_train]
target_row_train = target_row[index_train]
inputs_row_tests = inputs_row[index_tests]
inputs_col_tests = inputs_col[index_tests]
target_row_tests = target_row[index_tests]

## Model

### Parameters

#### Weights and bias

In [142]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#### Convolution and pooling

In [143]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [144]:
def rw_pool(x):
    U_max = tf.reduce_max(U, axis=1)
    U_tle = tf.tile(U_max,[3])
    U_shp = tf.reshape(U_tle,[3,3])
    return tf.Variable(U_shp)

def cw_pool(x):
    U_max = tf.reduce_max(U, axis=0)
    U_tle = tf.tile(U_max,[3])
    U_shp = tf.reshape(U_tle,[3,3])
    return tf.Variable(U_shp)

### Input and target

In [145]:
x_row = tf.placeholder(tf.float32, shape=[None, 3, 3, 2])
x_col = tf.placeholder(tf.float32, shape=[None, 3, 3, 2])
y_ = tf.placeholder(tf.float32, shape=[None, 3, 1])

### Hidden layer 1 (row player)

In [146]:
W_conv1 = weight_variable([1, 1, 2, 50])
b_conv1 = bias_variable([50])
x_image = tf.reshape(x_row, [-1, 3, 3, 2])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

### Hidden layer 2 (row player)

In [147]:
W_conv2 = weight_variable([1, 1, 50, 50])
b_conv2 = bias_variable([50])
h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)

In [148]:
keep_prob = tf.placeholder(tf.float32)
h_conv2_drop = tf.nn.dropout(h_conv2, keep_prob)

<div style="text-align: right"> <h3> Hidden layer 1 (col player) </h3> </div>

In [149]:
W_conv1_col = weight_variable([1, 1, 2, 50])
b_conv1_col = bias_variable([50])
x_image_col = tf.reshape(x_col, [-1, 3, 3, 2])
h_conv1_col = tf.nn.relu(conv2d(x_image_col, W_conv1_col) + b_conv1_col)

<div style="text-align: right"> <h3> Hidden layer 2 (col player) </h3> </div>

In [150]:
W_conv2_col = weight_variable([1, 1, 50, 50])
b_conv2_col = bias_variable([50])
h_conv2_col = tf.nn.relu(conv2d(h_conv1_col, W_conv2_col) + b_conv2_col)

In [151]:
keep_prob_col = tf.placeholder(tf.float32)
h_conv2_col_drop = tf.nn.dropout(h_conv2_col, keep_prob_col)

<div style="text-align: right"> <h3> Action response layer 0 (col player) </h3> </div>

In [152]:
ar_rwsm0_col = tf.reduce_sum(h_conv2_col_drop, axis=2)
ar_sfmx0_col = tf.nn.softmax(ar_rwsm0_col, dim=1)
ar_sfmx0_col = tf.slice(ar_sfmx0_col, [0, 0, 0], [-1, 3, 1])

### Action response layer 1 (row player response to col player)

In [153]:
ar_rdsm1 = tf.reduce_sum(h_conv2, axis=3)
ar_dtpt1 = tf.matmul(ar_rdsm1, ar_sfmx0_col)
y = tf.nn.softmax(ar_dtpt1, dim=1)

### Cost function

In [154]:
beta = 0.01
regularizer = tf.nn.l2_loss(W_conv1) + tf.nn.l2_loss(W_conv2) + tf.nn.l2_loss(W_conv1_col) + tf.nn.l2_loss(W_conv2_col)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y) + beta * regularizer, reduction_indices=[1]))

### Optimization

In [155]:
train_step = tf.train.AdamOptimizer(0.0002,0.9,0.999,1e-8).minimize(cross_entropy)

## Train

In [156]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

In [157]:
for i in range(2000):
    if i%5 == 0:
        shuffle = np.random.permutation(range(inputs_row_train.shape[0]))
    inputs_row_train_batch = inputs_row_train[shuffle[shuffle.shape[0]//5*(i%5):shuffle.shape[0]//5*(i%5+1)]]
    inputs_col_train_batch = inputs_col_train[shuffle[shuffle.shape[0]//5*(i%5):shuffle.shape[0]//5*(i%5+1)]]
    target_row_train_batch = target_row_train[shuffle[shuffle.shape[0]//5*(i%5):shuffle.shape[0]//5*(i%5+1)]]
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
                x_row: inputs_row_train_batch,
                x_col: inputs_col_train_batch,
                y_: target_row_train_batch, 
                keep_prob: 1.0,
                keep_prob_col: 1.0})
        train_NLL = cross_entropy.eval(feed_dict={
                x_row: inputs_row_train_batch, 
                x_col: inputs_col_train_batch,
                y_: target_row_train_batch,
                keep_prob: 1.0,
                keep_prob_col: 1.0})
        print("step %d, train accuracy %g, train NLL %g"%(i, train_accuracy, train_NLL))
    train_step.run(feed_dict={x_row: inputs_row_train_batch, 
                              x_col: inputs_col_train_batch, 
                              y_: target_row_train_batch, 
                              keep_prob: 0.8,
                              keep_prob_col: 0.8})

step 0, train accuracy 0.0625, train NLL 0.538047
step 100, train accuracy 0.6875, train NLL 0.0714749
step 200, train accuracy 0.8125, train NLL -0.14418
step 300, train accuracy 0.875, train NLL -0.531959
step 400, train accuracy 0.875, train NLL -0.929722
step 500, train accuracy 0.9375, train NLL -1.49675
step 600, train accuracy 0.9375, train NLL -2.05906
step 700, train accuracy 0.9375, train NLL -2.82296
step 800, train accuracy 0.9375, train NLL -3.67122
step 900, train accuracy 0.875, train NLL -4.6663
step 1000, train accuracy 0.9375, train NLL -5.73751
step 1100, train accuracy 0.875, train NLL -6.91006
step 1200, train accuracy 0.875, train NLL -8.23999
step 1300, train accuracy 0.9375, train NLL -9.60137
step 1400, train accuracy 1, train NLL -11.1477
step 1500, train accuracy 0.8125, train NLL -12.572
step 1600, train accuracy 0.9375, train NLL -14.3489
step 1700, train accuracy 0.9375, train NLL -16.3377
step 1800, train accuracy 0.875, train NLL -18.1792
step 1900, trai

## Test

In [158]:
test_accuracy = accuracy.eval(feed_dict={
        x_row: inputs_row_tests,
        x_col: inputs_col_tests,
        y_: target_row_tests, 
        keep_prob: 1.0,
        keep_prob_col: 1.0})
test_NLL = cross_entropy.eval(feed_dict={
        x_row: inputs_row_tests,
        x_col: inputs_col_tests,
        y_: target_row_tests, 
        keep_prob: 1.0,
        keep_prob_col: 1.0})
print("test accuracy %g, test NLL %g"%(test_accuracy, test_NLL))

test accuracy 0.875, test NLL -22.3762


In [159]:
pd.set_option('display.max_rows', None)
compare_y = y.eval(feed_dict={
                    x_row: inputs_row_tests, 
                    x_col: inputs_col_tests, 
                    y_: target_row_tests, 
                    keep_prob: 1.0, 
                    keep_prob_col: 1.0})
compare_y = compare_y.reshape(compare_y.shape[0], compare_y.shape[1])
compare_y_ = y_.eval(feed_dict={
                        x_row: inputs_row_tests,
                        x_col: inputs_col_tests,
                        y_: target_row_tests, 
                        keep_prob: 1.0,
                        keep_prob_col: 1.0})
compare_y_ = compare_y_.reshape(compare_y_.shape[0], compare_y_.shape[1])
compare = pd.DataFrame(np.concatenate((compare_y, compare_y_), axis=1))
compare.columns = ["y1","y2","y3","y_1","y_2","y_3"]
compare

,y1,y2,y3,y_1,y_2,y_3
0,0.302832,0.102956,0.594212,0.021277,0.042553,0.936170
1,0.220786,0.742158,0.037056,0.226667,0.693333,0.080000
2,0.808737,0.152558,0.038705,0.808511,0.191489,0.000000
3,0.808737,0.152558,0.038705,0.610000,0.350000,0.040000
4,0.221601,0.030962,0.747438,0.541667,0.062500,0.395833
5,0.220786,0.742158,0.037056,0.191489,0.617021,0.191489
6,0.002989,0.703813,0.293198,0.013333,0.546667,0.440000
7,0.396428,0.229226,0.374346,0.489796,0.190476,0.319728
8,0.347797,0.633851,0.018353,0.442177,0.517007,0.040816
9,0.174303,0.678877,0.146819,0.142857,0.802721,0.054422
